# Model building

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

import seaborn as sns

from tools.helper_functions import moving_average, normalize, rolling_normal
from tools.data_loader import load_flight_data, select_sequence, clean_data, apply_motor_calibration

import tools.data_loader as dl

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures

from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from scipy.integrate import cumulative_trapezoid, simpson, romb
        
import glob
import os

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=160)

In [ ]:
# declare ALL list objects necessary

v_induced = 14.345219306215128 # induced velocity for firefly vehicle
markersize= 15 # global markersize setting for scatter plots

# set colors

lowCol = '#FF7518' # pumpkin orange
upCol = '#18A2FF' # complementary color

upMeanCol = 'blue'
lowMeanCol = 'black'

armCol = upCol
armMeanCol = 'red'

## Load flight data

In [ ]:
# select test flight

# first testflight
#flight = f'../flight_data/2022-08-01_ag_field/flight_6_first_hover_flight/'
#hover_start = 35
#hover_end = 15

# second testflight
flight = f'../flight_data/2022-08-01_ag_field/flight_8_second_hover_flight/'
hoverStart = 30
hoverEnd = 190

# third testflight
flight2 = f'../flight_data/2022-08-29_ag_field/flight_2_delta0_sweep/'
hoverStart2 = 100
hoverEnd2 = 510

In [ ]:
fd = load_flight_data(flight)
fd2 = load_flight_data(flight2)

fd = dl.convert_time(fd)
fd2 = dl.convert_time(fd2)

## Preprocessing

### Rename columns and calculate power values for motors, arms and vehicle

In [ ]:
fd = clean_data(fd)
fd2 = clean_data(fd2)

### Apply calibration for ESC current and voltage

In [ ]:
fd = apply_motor_calibration(fd)
fd2 = apply_motor_calibration(fd2)

In [ ]:
# Perform regression RPM vs. current

rpm_range = np.linspace(0, 3000, 100).reshape(-1,1)
currentRegMotorValues = []
currentRegCurve = []
fd_current = []


for i in range(1,9):
    
    # get all points where I3 has its minimum value and RPM is greater than 0
    filtered = fd[fd[f'I{i}']> fd[f'I{i}'].min()]
    filtered = filtered[filtered[f'rpm{i}']> 0]
    
    rpm = filtered[[f'rpm{i}']]
    current = filtered[[f'I{i}']]
    
    rpmAsPolynomial = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm)
    
    currentRegression = LinearRegression(fit_intercept=False, positive=True).fit(rpmAsPolynomial, current)

    rpmRangePolynomial = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm_range)
    
    currentPredictions = currentRegression.predict(rpmRangePolynomial)
    
    currentRegMotorValues.append(currentPredictions)
    fd_current.append(filtered)
    currentRegCurve.append(currentRegression)

In [ ]:
# correct values for current by using regression model

# save value of flight data DataFrame
#fd_corrected = fd.copy()
fd_old = fd.copy(deep=True)

# get indices where I3 is minimum
minI3 = fd['I3']==fd[f'I3'].min()
rpm3Data = fd['rpm3'][minI3].values.reshape(-1,1)

# transform polynomial features
rpm3Features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm3Data)

# perform regression
rpm3RegressionResult = currentRegCurve[2].predict(rpm3Features).reshape(-1)

# apply correction to current signal
fd.loc[minI3,'I3'] = rpm3RegressionResult

### Calculate power and controls

In [ ]:
fd = dl.calculate_power_and_rpm(fd)
fd2 = dl.calculate_power_and_rpm(fd2)

### Calculate motor commands

In [ ]:
fd = dl.calculate_motor_cmds(fd)
fd2 = dl.calculate_motor_cmds(fd2)

### Filter out hover sequence DataFrame

In [ ]:
# Reset current correction
hoverIndex1, fd_hover1 = select_sequence(fd, hoverStart, hoverEnd)
hoverIndex2, fd_hover2 = select_sequence(fd2, hoverStart2, hoverEnd2)

### Remove values after $\Delta_0$-step

In [ ]:
steps1 = fd_hover1[np.abs(fd_hover1['delta0'].diff()) > 0]
steps2 = fd_hover2[np.abs(fd_hover2['delta0'].diff()) > 0]

for i in steps1.index:
    toDrop = np.arange(i,i+20)
    fd_hover1 = fd_hover1.drop(index=toDrop)
    
for i in steps2.index:
    toDrop = np.arange(i,i+20)
    fd_hover2 = fd_hover2.drop(index=toDrop)

### Concatenate two flights

In [ ]:
fd_hover = pd.concat([fd_hover1, fd_hover2])
fd_hover['t'] = np.arange(0,len(fd_hover)/30, 1/30)

## Signal filtering

### Apply filter so individual signals

In [ ]:
# Total power over time

fig0, ax0 = plt.subplots(2,2, figsize=(10,8))
fig0.suptitle('Comparison of power and velocity signal', fontsize=14)

# Rotor pairs
a = 0
b = -1
step = 1

cm = 'viridis'

# x-position
ax0[0][0].set_title(f'x-Position over time')
ax0[0][0].plot(fd_hover1['t'], fd_hover1['w'], label='u-Signal')
ax0[1][0].plot(fd_hover2['t'], fd_hover2['w'], label='u-Signal')

# y-position
ax0[0][1].set_title(f'Power over time')

ax0[0][1].plot(fd_hover1['t'], fd_hover1['pVehicle'] - fd_hover1['pVehicle'].mean(), label='power-Signal')
ax0[1][1].plot(fd_hover2['t'], fd_hover2['pVehicle'] - fd_hover2['pVehicle'].mean(), label='power-Signal')

for idx in range(4):
    
    i = int(idx/2)
    j = idx%2
    
    ax0[i][j].grid()
    ax0[i][j].set_xlabel('Time [s]')
    ax0[i][j].legend()

ax0[0][0].set_ylabel('Velocity [m/s]')
ax0[0][1].set_ylabel('Power [W]')
ax0[1][0].set_ylabel('[-]')
ax0[1][1].set_ylabel('[-]')

fig0.tight_layout()

# Regression model for power

## Linear model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
fd_hover['pMo1'].mean()

In [ ]:
fd_hover1['delta0^2'] = fd_hover1['delta0'] ** 2
fd_hover2['delta0^2'] = fd_hover2['delta0'] ** 2
fd_hover['delta0^2'] = fd_hover['delta0'] ** 2

In [ ]:
# assemble features and y-values
featureSet1 = []
featureSet2 = []
featureSet3 = []

output = []

for i in range(1,9):
    featureSet1.append(f'uOut{i}')
    featureSet2.append(f'uOut{i}')
    featureSet3.append(f'uOut{i}')
    output.append(f'pMo{i}')

# add features to set 1
featureSet1.append('delta0^2')

# add features to set 2 
featureSet2.append('delta0^2')
#featureSet2.append('z')

# add features to set 3
featureSet3.append('delta0^2')
featureSet3.append('p')
featureSet3.append('q')

X1, X1_test, Y1, Y1_test = train_test_split(fd_hover1[featureSet1].copy(deep=True),
                                            fd_hover1[output].copy(deep=True),
                                            test_size=0.2, random_state=42)

X2, X2_test, Y2, Y2_test = train_test_split(fd_hover2[featureSet2].copy(deep=True),
                                            fd_hover2[output].copy(deep=True),
                                            test_size=0.2, random_state=42)

X3, X3_test, Y3, Y3_test = train_test_split(fd_hover[featureSet3].copy(deep=True),
                                            fd_hover[output].copy(deep=True),
                                            test_size=0.2, random_state=42)

X1_full = fd_hover1[featureSet1].copy(deep=True)
X2_full = fd_hover2[featureSet2].copy(deep=True)
X3_full = fd_hover[featureSet3].copy(deep=True)

Y1_full = fd_hover1[output].copy(deep=True)
Y2_full = fd_hover2[output].copy(deep=True)
Y3_full = fd_hover[output].copy(deep=True)

In [ ]:
# Perform multiple linear regression
powerRegression1 = LinearRegression().fit(X1, Y1)

powerRegression2 = LinearRegression().fit(X2, Y2)

powerRegression3 = LinearRegression().fit(X3, Y3)

In [ ]:
systemMatrix1 = powerRegression1.coef_.round(2)
systemMatrix2 = powerRegression2.coef_.round(2)
systemMatrix3 = powerRegression3.coef_.round(2)

bias1 = powerRegression1.intercept_
bias2 = powerRegression2.intercept_
bias3 = powerRegression3.intercept_

In [ ]:
print('System maxtrix: Flight 1\n')
print(f'{systemMatrix1}')

In [ ]:
print('System maxtrix: Flight 2\n')

print(f'{systemMatrix2}')

In [ ]:
print('System maxtrix: Combined\n')
print(f'{systemMatrix3}')

In [ ]:
print(f'{bias1}')

In [ ]:
print(f'{bias2}')

In [ ]:
print(f'{bias3}')

In [ ]:
yPredictionsTrain1 = X1.dot(systemMatrix1.T) + bias1
yPredictionsTrain2 = X2.dot(systemMatrix2.T) + bias2
yPredictionsTrain3 = X3.dot(systemMatrix3.T) + bias3

yPredictionsTest1 = X1_test.dot(systemMatrix1.T) + bias1
yPredictionsTest2 = X2_test.dot(systemMatrix2.T) + bias2
yPredictionsTest3 = X3_test.dot(systemMatrix3.T) + bias3

yPredictionsFull1 = X1_full.dot(systemMatrix1.T) + bias1
yPredictionsFull2 = X2_full.dot(systemMatrix2.T) + bias2
yPredictionsFull3 = X3_full.dot(systemMatrix3.T) + bias3

yPredictionsTrain1.columns = [f'pMo{i}' for i in range(1,9)]
yPredictionsTrain2.columns = [f'pMo{i}' for i in range(1,9)]
yPredictionsTrain3.columns = [f'pMo{i}' for i in range(1,9)]

yPredictionsTest1.columns = [f'pMo{i}' for i in range(1,9)]
yPredictionsTest2.columns = [f'pMo{i}' for i in range(1,9)]
yPredictionsTest3.columns = [f'pMo{i}' for i in range(1,9)]

yPredictionsFull1.columns = [f'pMo{i}' for i in range(1,9)]
yPredictionsFull2.columns = [f'pMo{i}' for i in range(1,9)]
yPredictionsFull3.columns = [f'pMo{i}' for i in range(1,9)]

In [ ]:
# calculate prediction error

predictionErrorTrain1 = Y1 - yPredictionsTrain1
predictionErrorTrain2 = Y2 - yPredictionsTrain2
predictionErrorTrain3 = Y3 - yPredictionsTrain3

predictionErrorTest1 = Y1_test - yPredictionsTest1
predictionErrorTest2 = Y2_test - yPredictionsTest2
predictionErrorTest3 = Y3_test - yPredictionsTest3

predictionErrorFull1 = Y1_full - yPredictionsFull1
predictionErrorFull2 = Y2_full - yPredictionsFull2
predictionErrorFull3 = Y3_full - yPredictionsFull3

predictionErrorTrainMean1 = predictionErrorTrain1.rolling(30,1).mean()
predictionErrorTrainMean2 = predictionErrorTrain2.rolling(30,1).mean()
predictionErrorTrainMean3 = predictionErrorTrain3.rolling(30,1).mean()

predictionErrorTestMean1 = predictionErrorTest1.rolling(30,1).mean()
predictionErrorTestMean2 = predictionErrorTest2.rolling(30,1).mean()
predictionErrorTestMean3 = predictionErrorTest3.rolling(30,1).mean()

predictionErrorFullMean1 = predictionErrorFull1.rolling(30,1).mean()
predictionErrorFullMean2 = predictionErrorFull2.rolling(30,1).mean()
predictionErrorFullMean3 = predictionErrorFull3.rolling(30,1).mean()

# mean absolute error
MAE1 = np.around(mean_absolute_error(Y1, yPredictionsTrain1, multioutput='raw_values'),2)
MAE2 = np.around(mean_absolute_error(Y2, yPredictionsTrain2, multioutput='raw_values'),2)
MAE3 = np.around(mean_absolute_error(Y3, yPredictionsTrain3, multioutput='raw_values'),2)

MAE1_test = np.around(mean_absolute_error(Y1_test, yPredictionsTest1, multioutput='raw_values'),2)
MAE2_test = np.around(mean_absolute_error(Y2_test, yPredictionsTest2, multioutput='raw_values'),2)
MAE3_test = np.around(mean_absolute_error(Y3_test, yPredictionsTest3, multioutput='raw_values'),2)

MAE1_full = np.around(mean_absolute_error(Y1_full, yPredictionsFull1, multioutput='raw_values'),2)
MAE2_full = np.around(mean_absolute_error(Y2_full, yPredictionsFull2, multioutput='raw_values'),2)
MAE3_full = np.around(mean_absolute_error(Y3_full, yPredictionsFull3, multioutput='raw_values'),2)

In [ ]:
print('MAE training flight 1: ' + str(MAE1))
print('MAE training flight 2: ' + str(MAE2))
print('MAE training combined: ' + str(MAE3))

print("")

print('MAE test flight 1: ' + str(MAE1_test))
print('MAE test flight 2: ' + str(MAE2_test))
print('MAE test combined: ' + str(MAE3_test))

print("")

print('MAE total flight 1: ' + str(MAE1_full))
print('MAE total flight 2: ' + str(MAE2_full))
print('MAE total combined: ' + str(MAE3_full))

In [ ]:
# plot power over time

fig2, ax2 = plt.subplots(4, 2, figsize=(10,16))
fig2.suptitle('Coparison between predicted and true motor power', fontsize=14, y=1.05)

for idx in range(8):
    
    i = int(idx/2)
    j = idx % 2
    
    ax2[i][j].plot(fd_hover1['t'], predictionErrorFull1[f'pMo{idx+1}'])
    ax2[i][j].plot(fd_hover1['t'], predictionErrorFullMean1[f'pMo{idx+1}'])

    ax2[i][j].set_title(f'Prediction error motor {idx+1}')
    
    ax2[i][j].set_ylim(-50,50)
    ax2[i][j].set_xlabel('time [s]')
    ax2[i][j].set_ylabel('Prediction error [W]')
    ax2[i][j].grid()

fig2.tight_layout()

In [ ]:
# plot power over time

fig3, ax3 = plt.subplots(4, 2, figsize=(10,16))
fig3.suptitle('Coparison between predicted and true motor power', fontsize=14)

for idx in range(8):
    
    i = int(idx/2)
    j = idx % 2
    
    ax3[i][j].plot(fd_hover2['t'], predictionErrorFull2[f'pMo{idx+1}'])
    ax3[i][j].plot(fd_hover2['t'], predictionErrorFullMean2[f'pMo{idx+1}'])

    ax3[i][j].set_title(f'Prediction error motor {idx+1}')
    
    ax3[i][j].set_ylim(-100,100)
    ax3[i][j].set_xlabel('time [s]')
    ax3[i][j].set_ylabel('Prediction error [%]')
    
    ax3[i][j].grid()

fig3.tight_layout()

In [ ]:
# plot power over time

fig4, ax4 = plt.subplots(4, 2, figsize=(10,16))
fig4.suptitle('Coparison between predicted and true motor power', fontsize=14)

for idx in range(8):
    
    i = int(idx/2)
    j = idx % 2
    
    ax4[i][j].plot(fd_hover['t'], predictionErrorFull3[f'pMo{idx+1}'])
    ax4[i][j].plot(fd_hover['t'], predictionErrorFullMean3[f'pMo{idx+1}'])

    ax4[i][j].set_title(f'Prediction error motor {idx+1}')
    
    ax4[i][j].set_ylim(-100,100)
    ax4[i][j].set_xlabel('time [s]')
    ax4[i][j].set_ylabel('Prediction error [%]')
    
    ax4[i][j].grid()

fig3.tight_layout()

In [ ]:
fdHover = fd_hover.copy(deep=True)

In [ ]:
fdHover.update(predictionErrorFull3)

In [ ]:
predictionErrorFull3

In [ ]:
fdHover.corr()['pMo2'].abs().sort_values(ascending=False).head(60)

-> Correlated with other CCW motors

In [ ]:
fdHover.corr()['pVehicle'].abs().sort_values(ascending=False).head(60)

In [ ]:
fdHover.columns